# tem Beneficios a entrega contínua?

## Revisar literatura para encontrar melhores perguntas de pesquisa

## Analisar dataset para estabelecer quais projetos tem entrega contínua



In [112]:
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId
client = MongoClient()
db = client.smartshark_2_2

### 1) Tem integração contínua - OK

In [113]:
vcs_systems = pd.DataFrame(list(db.vcs_system.find()))
travisBuilds = pd.DataFrame(list(db.travis_build.find()))
projetosComBuilds = travisBuilds['vcs_system_id'].unique()
vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()

,_id,url,project_id,last_updated,repository_type,repository_file
0,5bef24dc35e3ea2b7bc08208,https://github.com/apache/commons-rdf.git,5beebd870da3ef21a62b756e,2018-11-16 21:13:16.874,git,5dcbf72e2ecf182bf5d3ab89
2,58bfd79dfa94f2c7f0d32551,https://github.com/apache/commons-math,58bfd5860da3ef7cef464c78,2018-02-15 11:35:53.876,git,5dcbf5ce2ecf182bf5d37773
3,5b1fa04730a71b06bc3734c1,https://github.com/apache/mahout,5b1f9fbb0da3ef49b7d0a480,2018-06-12 12:28:23.953,git,5dcbf61c2ecf182bf5d382c4
4,5b27c64830a71b06bceb0ead,https://github.com/apache/opennlp.git,5b27c4cc0da3ef789daec0a1,2018-06-18 16:48:40.415,git,5dcbf6262ecf182bf5d3852f
5,5b34a9c830a71b06bc3d9ef3,https://github.com/apache/calcite.git,5b34a9210da3ef789cc240ec,2018-06-28 11:26:32.261,git,5dcbf6362ecf182bf5d38708
6,5b3b305530a71b06bc5bba5b,https://github.com/apache/cayenne.git,5b3b2d520da3ef789ec953ea,2018-07-03 10:14:13.878,git,5dcbf63c2ecf182bf5d387d8
8,5da726db3dfe5ddb85120d96,https://github.com/apache/bigtop.git,5da724f20da3ef03c84bcf48,2019-10-16 16:19:07.019,git,5dcbf7d72ecf182bf5d3c55d
9,5c87a755504acf99a4dbf833,https://github.com/apache/commons-bcel.git,5c87a7300da3ef6c8a4bf6d8,2019-03-12 13:34:29.466,git,5dcbf7602ecf182bf5d3b19e
10,5bed279735e3ea2b7b9d059e,https://github.com/apache/commons-beanutils.git,5bed276c0da3ef21a58b337a,2018-11-15 09:00:23.400,git,5dcbf7172ecf182bf5d3a90b
11,5bed5a4635e3ea2b7ba10f97,https://github.com/apache/commons-codec.git,5bed58010da3ef21a58b337b,2018-11-15 12:36:38.541,git,5dcbf7182ecf182bf5d3a91d


### 2) Pegar as tags do projetos de integração contínua? - OK

In [114]:
tags = pd.DataFrame(list(db.tag.find()))
idsVcsComBuilds = vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()._id

In [115]:
tagsDosProjComBuild = tags.query('vcs_system_id.isin(@idsVcsComBuilds)').drop_duplicates().dropna()

In [116]:
tagsDosProjComBuild

,_id,name,commit_id,vcs_system_id,message,tagger_id,date,date_offset
3,5bef24e783ad2f56e01ca88c,0.3.0-incubating,5bef24e783ad2f56e01ca88a,5bef24dc35e3ea2b7bc08208,Apache Commons RDF 0.3.0-incubating\n\nhttps:/...,58c1133102ca40f8bfb1e2a9,2016-11-15 13:28:17,0.0
5,5bef254783ad2f56d01caefd,0.1.0-incubating,5bef254683ad2f56d01caefb,5bef24dc35e3ea2b7bc08208,[maven-release-plugin] copy for tag 0.1.0-incu...,58bfd1cb02ca40f8bf1485a4,2015-05-07 14:10:41,120.0
27,58bfd7a53cff546cd3b42b9e,MATH_3_4_1_RC1,58bfd7a53cff546cd3b42b9a,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4.1 RC1 tag.\n...,58bfd79d3cff546cc7b42a28,2015-01-08 16:51:54,60.0
28,58bfd7a53cff546cd8b42c1b,MATH_3_4_RC2,58bfd7a53cff546cd8b42c17,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4 RC2 tag.\n--...,58bfd79d3cff546cc7b42a28,2014-12-19 14:41:18,60.0
29,58bfd7b83cff546cc8b43dab,MATH_1_2,58bfd7b83cff546cc8b43da7,58bfd79dfa94f2c7f0d32551,MATH_1_2\n,58bfd7a53cff546ccbb42b96,2008-02-24 05:56:08,0.0
...,...,...,...,...,...,...,...,...
4780,615e91666511689f63ec2c87,qpid-jms-0.1.0-beta,615e91666511689f63ec2c85,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag qpid-jms-0...,5a031143efa7b46cc724c03b,2015-03-09 18:24:25,0.0
4781,615e91ae6511689f63ec816c,1.2.0,615e91ae6511689f63ec816a,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.2.0\n,5e71cec61c4f2a5a2b6418d7,2021-09-24 12:36:32,60.0
4782,615e91af6511689f63ec8221,1.1.0,615e91af6511689f63ec821f,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.1.0\n,5f27cd89af02e2d6de7c7cdc,2021-07-05 15:35:44,60.0
4783,615e91b06511689f63ec8294,1.0.0,615e91b06511689f63ec8292,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.0.0\n,5f27cd89af02e2d6de7c7cdc,2021-05-19 14:58:57,60.0


### 3) Determinei oq vou considerar Tag de Release - OK
##### Tags de release não tem no nome os termos 'RC', 'candidate', 'test', 'beta', 'alfa', 'unreleased', snapshot, siteupdate, freeze, sandbox, migration

In [117]:
tagsDeRelase = tagsDosProjComBuild[
    tagsDosProjComBuild['name']
    .str.contains('rc|RC|candidate|CANDIDATE|test|TEST|beta|BETA|alpha|ALPHA|unreleased|UNRELEASED|snapshot|SNAPSHOT|siteupdate|SITEUPDATE|freeze|FREEZE|sandbox|SANDBOX|migration|MIGRATION')==False].drop_duplicates().dropna()

In [118]:
tagsDeRelase.name.to_csv('tags.csv', index=False)

In [119]:
tagsDeRelase

,_id,name,commit_id,vcs_system_id,message,tagger_id,date,date_offset
3,5bef24e783ad2f56e01ca88c,0.3.0-incubating,5bef24e783ad2f56e01ca88a,5bef24dc35e3ea2b7bc08208,Apache Commons RDF 0.3.0-incubating\n\nhttps:/...,58c1133102ca40f8bfb1e2a9,2016-11-15 13:28:17,0.0
5,5bef254783ad2f56d01caefd,0.1.0-incubating,5bef254683ad2f56d01caefb,5bef24dc35e3ea2b7bc08208,[maven-release-plugin] copy for tag 0.1.0-incu...,58bfd1cb02ca40f8bf1485a4,2015-05-07 14:10:41,120.0
29,58bfd7b83cff546cc8b43dab,MATH_1_2,58bfd7b83cff546cc8b43da7,58bfd79dfa94f2c7f0d32551,MATH_1_2\n,58bfd7a53cff546ccbb42b96,2008-02-24 05:56:08,0.0
32,58bfd7c33cff546cd1b447ff,MATH_3_1,58bfd7c33cff546cd1b447fb,58bfd79dfa94f2c7f0d32551,MATH_3_1\n,58bfd7a53cff546ccab42b89,2012-12-23 18:09:22,0.0
33,58bfd7cd3cff546cd5b44d22,MATH_3_5,58bfd7cd3cff546cd5b44d1c,58bfd79dfa94f2c7f0d32551,RC2 becomes the 3.5 official version.\n-----BE...,58bfd79d3cff546cc7b42a28,2015-04-17 14:39:07,120.0
...,...,...,...,...,...,...,...,...
4779,615e91646511689f63ec2b38,qpid-jms-0.1.0,615e91646511689f63ec2b36,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag qpid-jms-0...,5a031143efa7b46cc724c03b,2015-03-16 18:45:53,0.0
4781,615e91ae6511689f63ec816c,1.2.0,615e91ae6511689f63ec816a,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.2.0\n,5e71cec61c4f2a5a2b6418d7,2021-09-24 12:36:32,60.0
4782,615e91af6511689f63ec8221,1.1.0,615e91af6511689f63ec821f,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.1.0\n,5f27cd89af02e2d6de7c7cdc,2021-07-05 15:35:44,60.0
4783,615e91b06511689f63ec8294,1.0.0,615e91b06511689f63ec8292,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.0.0\n,5f27cd89af02e2d6de7c7cdc,2021-05-19 14:58:57,60.0


### 4) Filtrar builds que tem commits de tag de release - OK

In [120]:
commitsDeRelease = tagsDeRelase.commit_id.unique()
buildsIntegrados = travisBuilds.query('commit_id.isin(@commitsDeRelease)')
stateIndesejados = ['failed', 'canceled']
BuildsValidos = buildsIntegrados.query('~state.isin(@stateIndesejados)')

In [121]:
BuildsValidos.state.value_counts()

passed     216
errored     48
Name: state, dtype: int64

### 5) Analisar quais projetos tem builds de entrega (34 no total) ok 

In [122]:
vcsComEntrega = BuildsValidos.vcs_system_id.unique()
vcs_systems.query('_id.isin(@vcsComEntrega)').drop_duplicates().dropna()

,_id,url,project_id,last_updated,repository_type,repository_file
0,5bef24dc35e3ea2b7bc08208,https://github.com/apache/commons-rdf.git,5beebd870da3ef21a62b756e,2018-11-16 21:13:16.874,git,5dcbf72e2ecf182bf5d3ab89
4,5b27c64830a71b06bceb0ead,https://github.com/apache/opennlp.git,5b27c4cc0da3ef789daec0a1,2018-06-18 16:48:40.415,git,5dcbf6262ecf182bf5d3852f
5,5b34a9c830a71b06bc3d9ef3,https://github.com/apache/calcite.git,5b34a9210da3ef789cc240ec,2018-06-28 11:26:32.261,git,5dcbf6362ecf182bf5d38708
6,5b3b305530a71b06bc5bba5b,https://github.com/apache/cayenne.git,5b3b2d520da3ef789ec953ea,2018-07-03 10:14:13.878,git,5dcbf63c2ecf182bf5d387d8
8,5da726db3dfe5ddb85120d96,https://github.com/apache/bigtop.git,5da724f20da3ef03c84bcf48,2019-10-16 16:19:07.019,git,5dcbf7d72ecf182bf5d3c55d
9,5c87a755504acf99a4dbf833,https://github.com/apache/commons-bcel.git,5c87a7300da3ef6c8a4bf6d8,2019-03-12 13:34:29.466,git,5dcbf7602ecf182bf5d3b19e
11,5bed5a4635e3ea2b7ba10f97,https://github.com/apache/commons-codec.git,5bed58010da3ef21a58b337b,2018-11-15 12:36:38.541,git,5dcbf7182ecf182bf5d3a91d
12,5bed25af35e3ea2b7b8bb711,https://github.com/apache/commons-collections.git,5bed254e0da3ef21a62b756d,2018-11-15 08:52:15.394,git,5dcbf7162ecf182bf5d3a8bb
13,5bed61cc35e3ea2b7b2a1805,https://github.com/apache/commons-compress.git,5bed583a0da3ef21a58b337c,2018-11-15 13:08:44.599,git,5dcbf71b2ecf182bf5d3a931
14,5bed83ca35e3ea2b7be8d7c9,https://github.com/apache/commons-configuratio...,5bed83440da3ef21a2107882,2018-11-15 15:33:46.623,git,5dcbf71d2ecf182bf5d3a999


### 6) Desses projetos quais tem uma quantidade relevante de entregas que permita a nossa análise (>3 Builds com integração) (21 projetos no total)

In [123]:
BuildsValidosPorCommit = BuildsValidos[['_id', 'vcs_system_id', 'commit_id', 'number', 'state', 'started_at', 'finished_at']].groupby(['commit_id'], as_index=False, sort=False).min()

In [124]:
vcsComCD = BuildsValidosPorCommit[['vcs_system_id','_id']].groupby(['vcs_system_id'], as_index=False).agg('count').sort_values(by=['_id'], ascending=False)
vcsComCD = tmp.query('_id > 2')
print(vcsComCD.count())
vcsComCD

_id    21
dtype: int64


,_id
vcs_system_id,
5bd0268235e3ea2b7bbfdbae,17
5b43737430a71b06bcbdd168,13
5b23df3c30a71b06bc01e16a,12
5b7ffe9530a71b06bc70c038,9
5b27c64830a71b06bceb0ead,7
5bed61cc35e3ea2b7b2a1805,6
5d8ccfb33dfe5ddb8590d841,6
5c3f0a7c621a9a77b35f3add,6
608b9b33c26a57681ebf90b6,5


### 7) Percorrer os projetos e coletar seus dados

In [125]:
vcsList= []
for index, row in vcsComCD.iterrows():
    vcsList.append(row.name)
vcsList

[ObjectId('5bd0268235e3ea2b7bbfdbae'),
 ObjectId('5b43737430a71b06bcbdd168'),
 ObjectId('5b23df3c30a71b06bc01e16a'),
 ObjectId('5b7ffe9530a71b06bc70c038'),
 ObjectId('5b27c64830a71b06bceb0ead'),
 ObjectId('5bed61cc35e3ea2b7b2a1805'),
 ObjectId('5d8ccfb33dfe5ddb8590d841'),
 ObjectId('5c3f0a7c621a9a77b35f3add'),
 ObjectId('608b9b33c26a57681ebf90b6'),
 ObjectId('5cd2806d504acf99a4cd728c'),
 ObjectId('5bbe0f4057674ee16793f245'),
 ObjectId('5c87a96c504acf99a4de5a18'),
 ObjectId('5d91bc213dfe5ddb85cc1e0e'),
 ObjectId('5bed83ca35e3ea2b7be8d7c9'),
 ObjectId('615e9105f73e2aa390c8a69b'),
 ObjectId('5b3b305530a71b06bc5bba5b'),
 ObjectId('5b76bead30a71b06bcadc156'),
 ObjectId('5c87a755504acf99a4dbf833'),
 ObjectId('5b47693d30a71b06bc04ba2e'),
 ObjectId('5bc5d34957674ee16701b68a'),
 ObjectId('5bee7d5b35e3ea2b7ba03bfa')]

Montar a primeira tabela, que para cada vcs_system_id, busca project_id, issues_system_id, data da adoção de CD, primeira build de CD, quantas builds antes e depois do CD, campo de análise manual se tem a CD suspeita

In [150]:
issue_system = pd.DataFrame(list(db.issue_system.find()))
project_ids = []
issues_system_ids = []
for vcs in vcsList:
    # Buscar informações sobre o projeto
    project_id = vcs_systems.query('_id == @vcs').iloc[0,2]    
    project_ids.append(project_id)

    # Informações sobre o issue system
    issues_system_id = issue_system.query('project_id == @project_id').iloc[0,1]
    issues_system_ids.append(issues_system_id)

    # Pegar o commit da build considerada como a build de adoção
    #BuildsValidosPorCommit.query('vcs_system_id == @a').sort_values(by=['started_at']).iloc[1,5]

    # Pegar a release com o commit de build de adoção

    # Montar dataframe
project_ids, issues_system_ids



([ObjectId('5bd022e30da3ef1a10738359'),
  ObjectId('5b43728b0da3ef07106a790f'),
  ObjectId('5b2225520da3ef3e52ece5e2'),
  ObjectId('5b7ffd4e0da3ef049126bdbd'),
  ObjectId('5b27c4cc0da3ef789daec0a1'),
  ObjectId('5bed583a0da3ef21a58b337c'),
  ObjectId('5d8cb4a10da3ef0b68077801'),
  ObjectId('5c3f09db0da3ef512137483f'),
  ObjectId('608b9b0227ad48a9ceb48654'),
  ObjectId('5cd27fe20da3ef671cda02e7'),
  ObjectId('5bbe0efb0da3ef0ddd6d593d'),
  ObjectId('5c87a9510da3ef6c89b69038'),
  ObjectId('5d91bbb20da3ef35cfd2f443'),
  ObjectId('5bed83440da3ef21a2107882'),
  ObjectId('615e90db3044794dcc96fe1e'),
  ObjectId('5b3b2d520da3ef789ec953ea'),
  ObjectId('5b76be770da3ef049126bdba'),
  ObjectId('5c87a7300da3ef6c8a4bf6d8'),
  ObjectId('5b4769050da3ef0712a85126'),
  ObjectId('5bc5cf050da3ef0de20a7284'),
  ObjectId('5bee7a9d0da3ef21a3a094b1')],
 [ObjectId('5bd022e30da3ef1a10738359'),
  ObjectId('5b43728b0da3ef07106a790f'),
  ObjectId('5b2225520da3ef3e52ece5e2'),
  ObjectId('5b7ffd4e0da3ef049126bdbd'),

### 8) Buscar as issues dos projetos analisados para contabilizar as metricas

### 9) Contabilizar métricas

In [152]:
BuildsValidosPorCommit.query('vcs_system_id == @a').sort_values(by=['started_at']).iloc[1,5]

,commit_id,_id,vcs_system_id,number,state,started_at,finished_at
50,5b437e9f6d2aba30574abddf,5f3fa6a870c422d0da222840,5b43737430a71b06bcbdd168,81,passed,2016-09-24 06:59:51,2016-09-24 07:13:21
51,5b437ae46d2aba30544aa805,5f3fa71d70c422d0da22287e,5b43737430a71b06bcbdd168,103,passed,2016-09-27 06:48:54,2016-09-27 07:03:14
52,5b437a2f6d2aba30534a8693,5f3fa76c70c422d0da2228a9,5b43737430a71b06bcbdd168,118,passed,2016-09-28 10:50:21,2016-09-28 11:07:13
53,5b437bec6d2aba30594a7392,5f3fb45570c422d0da223021,5b43737430a71b06bcbdd168,761,passed,2016-11-23 07:56:27,2016-11-23 08:06:10
54,5b437e9f6d2aba30514ab2ae,5f3fb4c870c422d0da22308e,5b43737430a71b06bcbdd168,798,passed,2016-11-26 11:06:03,2016-11-26 11:18:43
55,5b437e366d2aba304c4b1eb7,5f3fb4cb70c422d0da223091,5b43737430a71b06bcbdd168,799,passed,2016-11-26 11:13:34,2016-11-26 11:23:20
56,5b437f0b6d2aba30584ac999,5f3fd1cb70c422d0da223e96,5b43737430a71b06bcbdd168,2018,passed,2017-04-26 22:56:24,2017-04-26 23:07:11
57,5b437beb6d2aba30564a8309,5f3fdbd170c422d0da224311,5b43737430a71b06bcbdd168,2416,passed,2017-08-13 03:16:22,2017-08-13 03:24:09
58,5b437dfc6d2aba30534afe70,5f3fdfb670c422d0da224535,5b43737430a71b06bcbdd168,2649,passed,2017-10-29 08:08:07,2017-10-29 08:22:28
59,5b437b4f6d2aba30534aa6d7,5f3fe45570c422d0da2247fb,5b43737430a71b06bcbdd168,3012,passed,2018-03-01 03:01:31,2018-03-01 03:13:44


Seleciondo os 20 projetos criar uma tabela com vsc id, project_id e issue_system_id e data de corte (data da tag de entrega)

In [128]:
BuildsValidosPorCommit.event_type.value_counts()

AttributeError: 'DataFrame' object has no attribute 'event_type'

In [ ]:
issues = pd.DataFrame(list(db.issue.find()))

In [ ]:
issues

,_id,external_id,issue_system_id,title,desc,created_at,updated_at,creator_id,reporter_id,issue_type,...,resolution,fix_versions,assignee_id,issue_links,issue_type_manual,issue_type_verified,original_time_estimate,parent_issue_id,environment,is_pull_request
0,5bf43342ed024056f5b30676,COMMONSRDF-69,5bf432b8ed024056f5b30103,Update NOTICE year,As [~brunodepaulak@yahoo.com.br] commented dur...,2017-11-03 01:26:46.039,2017-12-06 13:17:08.002,5b6407a830a71b06bc9e5575,5b6407a830a71b06bc9e5575,Improvement,...,Resolved,[],5bc05c8c57674ee1678d9bcd,[],NaN,NaN,NaN,NaN,NaN,NaN
1,5bf43344ed024056f5b3067b,COMMONSRDF-73,5bf432b8ed024056f5b30103,"Jena module has Simple dependency, SPI will ne...",I'm trying to setup the RDF object one should ...,2017-11-23 11:06:41.323,2017-12-06 13:24:26.777,5b76a31530a71b06bcad0576,5b76a31530a71b06bcad0576,Bug,...,NaN,[],5bc05c8c57674ee1678d9bcd,[],NaN,NaN,NaN,NaN,NaN,NaN
2,5bf43346ed024056f5b30687,COMMONSRDF-71,5bf432b8ed024056f5b30103,RDF4J javadoc link is incorrect,When building the javadocs for commons-rdf-rdf...,2017-11-20 16:31:47.547,2018-01-30 04:31:25.538,5b868a5730a71b06bcd3d75e,5b868a5730a71b06bcd3d75e,Bug,...,Fixed,[0.6.0],5b6407a830a71b06bc9e5575,[],"{'sherbold': 'documentation', 'ftrautsch': 'do...",documentation,NaN,NaN,NaN,NaN
3,5bf43347ed024056f5b30693,COMMONSRDF-67,5bf432b8ed024056f5b30103,Make build compatible with JDK9,As pointed by [~acoburn] during the release of...,2017-11-03 00:58:34.178,2018-01-30 04:37:17.788,5b6407a830a71b06bc9e5575,5b6407a830a71b06bc9e5575,Improvement,...,Fixed,[0.6.0],5b6407a830a71b06bc9e5575,[],NaN,NaN,NaN,NaN,NaN,NaN
4,5bf43348ed024056f5b3069d,COMMONSRDF-74,5bf432b8ed024056f5b30103,Add Dataset and DatasetGraph conversions to Je...,Users will find it helpful to have conversions...,2018-01-29 20:15:30.423,2018-02-13 00:36:20.383,5bf4333a35e3ea2b7bdd9656,5bf4333a35e3ea2b7bdd9656,Improvement,...,Fixed,[0.6.0],5bc05c8c57674ee1678d9bcd,[],NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207395,5bc9a27d8dcf5070bc282dc3,PROTOCOLS-68,5bc9a2208dcf5070bc282b9f,Add sievemanager as protocols module,A Protocol for Remotely Managing Sieve Scripts...,2011-12-23 15:28:12.982,2017-03-14 06:09:41.216,5bc852bd57674ee167c8f978,5bc852bd57674ee167c8f978,Task,...,NaN,[2.0.0],NaN,"[{'issue_id': 5bc9a27d8dcf5070bc282dc2, 'type'...",NaN,NaN,NaN,NaN,NaN,NaN
207396,5bc9a27e8dcf5070bc282dcb,PROTOCOLS-73,5bc9a2208dcf5070bc282b9f,Move imap code into protocols,NaN,2011-12-23 21:09:53.240,2017-03-14 15:27:43.942,5bc852bd57674ee167c8f978,5bc852bd57674ee167c8f978,New Feature,...,NaN,[2.0.0],5bc852bd57674ee167c8f978,[],NaN,NaN,NaN,NaN,NaN,NaN
207397,5bc9a27e8dcf5070bc282dd4,PROTOCOLS-117,5bc9a2208dcf5070bc282b9f,"[IMAP] Delegated folders should appear in an ""...","When Alice shares its ""Shared folder"" with Bob...",2017-10-31 13:40:57.923,2017-10-31 13:40:57.923,5bc85cd257674ee167cdbd99,5bc85cd257674ee167cdbd99,Improvement,...,NaN,[],5bc8585157674ee167cb28d7,[],NaN,NaN,NaN,NaN,NaN,NaN
207398,5bc9a27f8dcf5070bc282dd5,PROTOCOLS-118,5bc9a2208dcf5070bc282b9f,Continuation Response can't handle Login,I was testing with the NGINX Imap Proxy and no...,2018-05-14 18:32:46.047,2018-06-11 18:30:47.742,5bc854c257674ee167c99ca5,5bc854c257674ee167c99ca5,Bug,...,NaN,[],5bc8585157674ee167cb28d7,[],NaN,NaN,NaN,NaN,Docker,NaN


In [ ]:
issues.issue_type.value_counts()

Bug                                      108137
Improvement                               49512
New Feature                               13216
Sub-task                                  11755
Task                                      11367
Dependency upgrade                         1779
Wish                                       1770
Test                                       1514
Story                                       978
Question                                    412
Documentation                               399
Epic                                        258
Technical task                               47
Brainstorming                                32
Temp                                         31
Umbrella                                     26
Dependency                                   19
Project                                       9
Request                                       7
Blog - New Blog Request                       6
New JIRA Project                        